In [1]:
from model.TagBert import TagBert

In [13]:
import pandas as pd
from tqdm import tqdm
import os

In [3]:
data_dir='datasets'
fname='meta_list.json'
path=os.path.join(data_dir,fname)

In [4]:
df=pd.read_json(path)
df=df[df['has_lyric']==1]

In [5]:
import torch
from transformers import BertModel
from kobert_tokenizer import KoBERTTokenizer

In [6]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [7]:
def encode(tokenizer,text):
    if isinstance(text,str):
        if text:
            text=[text]
    ret=[]
    for ele in text:
        ele=tokenizer.cls_token+ele+tokenizer.eos_token
        ele=tokenizer.encode(ele)
        ret.append(ele)
    return ret

In [8]:
def embed(model,encoded,device):
    if not isinstance(encoded,list):
        encoded=[encoded]
    ret=[]
    for ele in encoded:
        if len(ele) >512: 
            ele=ele[:512]
        token_ids=torch.tensor(ele,device=device).unsqueeze(0)
        output=model(token_ids)[0]
        output=output[:,0,:]
        ret.append(output)
    return ret

In [17]:
tagbert=TagBert(model,tokenizer,encode,embed,device='gpu')

Use gpu cuda


In [10]:
from utils.utils import DataUtils
from translator.Papago import Papago

client_id='A4Mceclx8PV864nLzpJA'
client_secret='gSGhYL3Roj'

translator=Papago(client_id,client_secret)
utils=DataUtils(translator)

In [11]:
excluded_tags=['NNB','NNM','VXV','VXA','VCP','VCN','MAG','MAC']
included_tags=['NNG','NNP']

In [19]:
for title,lyric in zip(df['track_title'].iloc[:5],df['lyric'].iloc[:5]):
    if utils.translator.get_language_type(lyric) == 'en': continue
    tags=tagbert.extract_tag(lyric,keyphrase_ngram_range=(1, 1), stop_words=None,top_n=20)
    tags=tags[0]
    words=list(map(lambda x:x[0],tags))
    probs=list(map(lambda x:x[1],tags))
    words=utils.extractMorph(words,included_tags)
    print(title)
    print(words)

밤편지
['마음속', '나의', '행운', '입맞춤', '사랑', '그날', '꺼', '반딧불', '눈', '위', '말']
모든 날, 모든 순간 (Every day, Every Moment)
['품속', '나의', '날', '유일한', '휴식처', '햇살', '하루']
첫눈처럼 너에게 가겠다
['숨결', '부', '순간', '질투', '축복', '빗물', '바람', '행복']
벚꽃 엔딩
['벚꽃', '잎', '자장 노래', '밤', '봄바람', '모습', '거리', '둘', '오예', '친구', '연인']
봄날
['영원', '원망', '말로', '반대편', '시간', '허공', '봄날', '그리움']


In [18]:
for title,lyric in zip(df['track_title'].iloc[:5],df['lyric'].iloc[:5]):
    if utils.translator.get_language_type(lyric) == 'en': continue
    tags=tagbert.extract_tag(lyric,keyphrase_ngram_range=(1, 2), stop_words=None,top_n=20)
    tags=tags[0]
    words=list(map(lambda x:x[0],tags))
    probs=list(map(lambda x:x[1],tags))
    words=utils.extractMorph(words,included_tags)
    print(title)
    print(words)

밤편지
['말', '반딧불', '파도', '글씨', '사랑 그날', '마음속', '입맞춤', '마음속', '그날 반딧불', '행운', '나의', '눈', '행운']
모든 날, 모든 순간 (Every day, Every Moment)
['눈물', '고단 나의', '미래 품속', '품속', '불안 나의', '품', '고단', '품속', '나의', '유일 휴식처', '날 햇살', '날', '눈빛 꿈', '눈물', '그날', '꿈']
첫눈처럼 너에게 가겠다
['삶', '부', '침', '숨결', '빗물', '한번쯤 행복', '숨결', '한번', '질투', '숨결']
벚꽃 엔딩
['자장 노래', '거리', '거리', '둘', '봄바람', '불면', '거리 둘', '봄바람', '둘', '거리 밤', '거리', '모습', '친구', '가요 오예', '오예 사랑', '벚꽃 잎']
봄날
['영원', '계절 영원', '그리움', '연기 말로', '눈꽃', '눈꽃', '봄날', '벚꽃', '그리움', '허공', '봄날', '사진', '겨울']
